In [14]:
from litellm import completion
import instructor
from pydantic import BaseModel
import ollama
from langchain_core.utils.function_calling import convert_to_openai_tool
from rich import print

In [15]:
client = instructor.from_litellm(completion)
ollama_client = ollama.Client(host = "http://localhost:11434")

In [19]:
def completion_with_tool_call(
    message : str
):
    class ChatResponse(BaseModel):
        chat : str
        
    def get_score(team : str):
        """Returns the score of a game."""
        return f"{team} won the game, 3-0"    
    
    tools = [
        convert_to_openai_tool(get_score)
    ]
    
    message = [{"role":  "user", "content": message}]
    
    completion_response = completion(
        model = "openai/llama3.1",
        base_url = "http://localhost:11434/v1",
        api_key = "ollama",
        messages = message,
        tools = tools 
    )
    ollama_response = ollama_client.chat(
        model = "llama3.1",
        messages = message,
        tools = tools
    )
    return completion_response, ollama_response

In [20]:
print(completion_with_tool_call("Hi, how are you?"))

(
    ModelResponse(
        id='chatcmpl-288',
        choices=[
            Choices(
                finish_reason='stop',
                index=0,
                message=Message(
                    content='I\'m here to provide information and assist with function calls. The prompt "how are 
you?" doesn\'t require any tool calling capabilities, so I will simply state that there is no specific JSON 
response for this question. However, if you\'d like to clarify or ask a yes/no type of question, I\'d be happy to 
help.',
                    role='assistant',
                    tool_calls=None,
                    function_call=None
                )
            )
        ],
        created=1723576406,
        model='llama3.1',
        object='chat.completion',
        system_fingerprint='fp_ollama',
        usage=Usage(completion_tokens=68, prompt_tokens=149, total_tokens=217),
        service_tier=None
    ),
    {
        'model': 'llama3.1',
        'created_at': '2024-08-13T19:13:27.826329161Z',
        'message': {
            'role': 'assistant',
            'content': '',
            'tool_calls': [{'function': {'name': 'get_score', 'arguments': {'team': 'YourMood'}}}]
        },
        'done_reason': 'stop',
        'done': True,
        'total_duration': 1429200033,
        'load_duration': 13145094,
        'prompt_eval_count': 149,
        'prompt_eval_duration': 8995000,
        'eval_count': 159,
        'eval_duration': 1364799000
    }
)

In [21]:
print(completion_with_tool_call("Did England win?"))

(
    ModelResponse(
        id='chatcmpl-115',
        choices=[
            Choices(
                finish_reason='tool_calls',
                index=0,
                message=Message(
                    content='',
                    role='assistant',
                    tool_calls=[
                        ChatCompletionMessageToolCall(
                            function=Function(arguments='{"team":"Manchester City"}', name='get_score'),
                            id='call_yj64lhaj',
                            type='function'
                        )
                    ],
                    function_call=None
                )
            )
        ],
        created=1723576417,
        model='llama3.1',
        object='chat.completion',
        system_fingerprint='fp_ollama',
        usage=Usage(completion_tokens=148, prompt_tokens=147, total_tokens=295),
        service_tier=None
    ),
    {
        'model': 'llama3.1',
        'created_at': '2024-08-13T19:13:38.529106117Z',
        'message': {
            'role': 'assistant',
            'content': '',
            'tool_calls': [
                {'function': {'name': 'get_score', 'arguments': {'team': 'England'}}},
                {'function': {'name': 'get_score', 'arguments': {'team': 'England'}}}
            ]
        },
        'done_reason': 'stop',
        'done': True,
        'total_duration': 785246353,
        'load_duration': 13213544,
        'prompt_eval_count': 147,
        'prompt_eval_duration': 9030000,
        'eval_count': 85,
        'eval_duration': 721537000
    }
)

In [30]:
def completion_with_agent_execution(
    inputs : str
):
    import json
    
    class ChatResponse(BaseModel):
        chat : str
        
    def get_score(team : str):
        """Returns the score of a game."""
        return f"{team} won the game, 3-0"    
    
    tools = [
        convert_to_openai_tool(get_score)
    ]
    
    message = [{"role":  "user", "content": inputs}]
    
    completion_response = completion(
        model = "openai/llama3.1",
        base_url = "http://localhost:11434/v1",
        api_key = "ollama",
        messages = message,
        tools = tools 
    )

    message.append(completion_response.choices[0].message)
    completion_messages = message

    if completion_response.choices[0].message.tool_calls:
        tool_calls = completion_response.choices[0].message.tool_calls
        
        available_functions = {
            "get_score" : get_score
        }
        
        for tool_call in tool_calls:
            completion_function_name = tool_call.function.name
            function_to_call = available_functions[completion_function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            completion_function_response = function_to_call(**function_args)
            print(completion_function_response)
            
    
    completion_messages.append(
        {
            "tool_call_id" : completion_response.choices[0].message.tool_calls[0].id,
            "role" : "tool",
            "name" : completion_function_name,
            "content" : completion_function_response
        }
    )
    second_completion_response = completion(
        model = "openai/llama3.1",
        base_url = "http://localhost:11434/v1",
        api_key = "ollama",
        messages = completion_messages,
        tools = tools
    )
    return second_completion_response

In [31]:
print(completion_with_agent_execution("Did England win?"))

England won the game, 3-0

ModelResponse(
    id='chatcmpl-700',
    choices=[
        Choices(
            finish_reason='stop',
            index=0,
            message=Message(
                content="The output was just a placeholder string. I don't know what game or tool call that output 
refers to. If you'd like to ask another question, I'll try to provide a better answer!",
                role='assistant',
                tool_calls=None,
                function_call=None
            )
        )
    ],
    created=1723577312,
    model='llama3.1',
    object='chat.completion',
    system_fingerprint='fp_ollama',
    usage=Usage(completion_tokens=41, prompt_tokens=71, total_tokens=112),
    service_tier=None
)